In [1]:
import pandas as pd
from dataset_manager import DatasetManager

# 1 - Load de dados

In [4]:
dataset_path = "./datasets"
manager = DatasetManager(dataset_path)
manager.show_datasets()

,compression,description,format,identifier,local_source,source
0,NaN,Quantidade de projetos para a construção de cr...,json,aprovacao_construcao_creche,./data/aprovacao_construcao_creche.json,http://api.pgi.gov.br/api/1/serie/2555.json
1,NaN,Educação Infantil e Ensino fundamental: número...,xls,data_rio_2004_2008,./data/data_rio_2004_2008.xls,./data/data_rio_2004_2008.xls
2,NaN,Quantidade de creches e pré-escolas cujas obra...,json,construcao_creche,./data/construcao_creche.json,http://api.pgi.gov.br/api/1/serie/2620.json
3,NaN,Valor do investimento para a construção das cr...,json,investimento_em_construcao_creche,./data/investimento_em_construcao_creche.json,http://api.pgi.gov.br/api/1/serie/1703.json
4,zip,Média de alunos por turmas em escolas e creche...,xls,media_alunos_2010,./data/media_alunos_2010,http://download.inep.gov.br/informacoes_estati...
5,zip,censo 2000 familias,xls,censo_rj_familias_2000,./data/censo_2000,ftp://ftp.ibge.gov.br/Censos/Censo_Demografico...
6,zip,Média de alunos por turmas em escolas e creche...,xls,media_alunos_2008,./data/media_alunos_2008,http://download.inep.gov.br/informacoes_estati...
7,zip,censo 2010 familias,xls,censo_rj_familias_2010,./data/censo_2010,ftp://ftp.ibge.gov.br/Censos/Censo_Demografico...
8,zip,Média de alunos por turmas em escolas e creche...,xls,media_alunos_2009,./data/media_alunos_2009,http://download.inep.gov.br/informacoes_estati...


In [7]:
manager.prepare_dataset()

05-18 08:50 - DatasetManager - prepare_dataset - INFO - Preparing aprovacao_construcao_creche ...
05-18 08:50 - DatasetManager - prepare_dataset - INFO - aprovacao_construcao_creche is ready to use!
05-18 08:50 - DatasetManager - prepare_dataset - INFO - Preparing data_rio_2004_2008 ...
05-18 08:50 - DatasetManager - prepare_dataset - INFO - data_rio_2004_2008 is ready to use!
05-18 08:50 - DatasetManager - prepare_dataset - INFO - Preparing construcao_creche ...
05-18 08:50 - DatasetManager - prepare_dataset - INFO - construcao_creche is ready to use!
05-18 08:50 - DatasetManager - prepare_dataset - INFO - Preparing investimento_em_construcao_creche ...
05-18 08:50 - DatasetManager - prepare_dataset - INFO - investimento_em_construcao_creche is ready to use!
05-18 08:50 - DatasetManager - prepare_dataset - INFO - Preparing media_alunos_2010 ...
05-18 08:50 - DatasetManager - prepare_dataset - INFO - media_alunos_2010 is ready to use!
05-18 08:50 - DatasetManager - prepare_dataset - IN

# 2 - Ler dataset de media de alunos

In [ ]:
datasets_keys = [x for x in manager.get_datasets() if x.endswith("construcao_creche")]

In [ ]:
import json

def read_dataset(identifier):
    dataset = manager.get_dataset(identifier)
    with open(dataset['local_source']) as local:
        json_string = json.load(local)['valores']
        return pd.DataFrame(json_string)

In [ ]:
aprovacao_construcao_creche = read_dataset(datasets_keys[0])
construcao_creche = read_dataset(datasets_keys[1])
investimento_em_construcao_creche = read_dataset(datasets_keys[2])

# 3 - Renomear colunas

In [ ]:
aprovacao_construcao_creche.columns = ["ano", "municipio_ibge", "aprovadas"]
aprovacao_construcao_creche.head()

In [ ]:
construcao_creche.columns = ["ano", "municipio_ibge", "construidas"]
construcao_creche.head()

In [ ]:
df = investimento_em_construcao_creche.merge(
    construcao_creche, on=["municipio_ibge", "ano"]
).merge(
    aprovacao_construcao_creche, on=["municipio_ibge", "ano"]
)

# 4 - Filtro

In [ ]:
def por_ano(df):
    df_anos = []
    anos = sorted(list(df[["ano"]].iloc[:,0].unique()))
    for ano in anos:
        df_anos.append(df[df["ano"] == ano])
    return df_anos

In [ ]:
def mais_gastaram(dfs):
    final_df = []
    for df in dfs:
        final_df.append(df.sort_values(["ano", "valor"], ascending=[False, False]).head())        
    return pd.concat(final_df)

In [ ]:
df_gastaram = mais_gastaram(por_ano(df))

In [ ]:
df_gastaram.head()

## 5 -  Cidades que mais gastaram por ano

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
sns.set_style("whitegrid")

In [ ]:
plt.subplots(figsize=(12,6))
plt.title("Cidades que mais gastaram construindo creches")

sns.barplot(x="municipio_ibge",
            y="valor",
            hue="ano",
            data=df_gastaram,
            ci=None)
plt.legend(loc=[1,1])


In [ ]:
plt.subplots(figsize=(12,6))
plt.title("Número de creches contruídas e valores")

sns.scatterplot(df_gastaram.municipio_ibge.apply(str),
            df_gastaram.construidas,
            s=df_gastaram.valor/100000, hue=df_gastaram.ano)
plt.show()

In [ ]:
plt.subplots(figsize=(12,6))
plt.title("Número de creches aprovadas e valores")

sns.scatterplot(df_gastaram.municipio_ibge.apply(str),
            df_gastaram.aprovadas,
            s=df_gastaram.valor/100000, hue=df_gastaram.ano)
plt.show()